Numpy treats a column vector differently from a 1 dimensional array. \
In order to consistently use column vectors, the following helper function takes either a numpy array or a numpy one-column
matrix(i.e. a column vector) and returns the data as a column vector.

In [1]:
import numpy as np
import pandas as pd

In [2]:
def as_colvec(x):
    if(x.ndim==2):
        return x
    else:
        return np.expand_dims(x,axis=1)

In [3]:
np.arange(4)

array([0, 1, 2, 3])

In [4]:
as_colvec(np.arange(4))

array([[0],
       [1],
       [2],
       [3]])

Recall that the first step in the BL procedure was to reverse engineer the implied returns vector $\pi$ from a set of portfolio weights $w$ 
$$ $$
$$ \pi = \sigma \sum {w}   $$
$$ $$
This is performed by the following code

In [5]:
def implied_returns(delta,sigma,w):
    '''
Obtain the implied expected returns by reverse engineering the weights
Inputs:
delta: Risk Aversion Coefficient (scalar)
sigma: Variance-Covariance Matrix (N x N) as DataFrame
    W: Portfolio weights (N x 1) as Series

Returns an N x 1 vector of Returns as Series 
    '''
    ir = delta * sigma.dot(w).squeeze() 
    # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

If the investor does not have a specific way to explicitly quantify the uncertaintly associated with the view in the $\Omega$ matrix, one could make the simplifying assumption that $\Omega$ is proportional to the variance of the prior.
$$ $$
$$ \Omega = diag(P(\tau \Sigma)P^{T}) $$

In [6]:
# Assumes that Omega is proportional to the variance of the prior
def proportional_prior(sigma,tau,p):
    '''
Returns that He-Litterman simplified Omega
Inputs:
sigma: N x N Covariance Matrix as DataFrame
tau  : a scalar
p    : a K x N DataFrame Linking Q and Assests
returns a P x P DataFrame, a Matrix representing Prior Uncertainties
    '''
    helit_omega = p.dot(tau*sigma).dot(p.T)
    # make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index,columns=p.index)
    

In [7]:
from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q, omega=None,delta=2.5,tau=0.02):
    '''
Computes the posterior expected returns based on the original black litterman reference model

W.prior must be an N x 1 vector of weights, a Series
Sigma.prior is an N x N covariance matrix, a DataFrame
P must be a K x N matrix linking Q and the Assests, a DataFrame
Q must be a K x 1 vector of views, a Series
Omega must be a K x K matrix a DataFrame, or None
if Omega is None, we assume it is
    proportional to variance of the prior
delta and tau are scalars    
    '''
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vector
    # How many assets do we have?
    N = w_prior.shape[0]
    
    # How many views?
    K = q.shape[0]
    
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior, w_prior)
    
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior
    
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require Omega to be inverted
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #    mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T)+omega).dot(q-p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
    #   sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T)+omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl,sigma_bl)

## Example

Consider the portfolio consisting of just two stocks : Intel (INTC) and Pfizer(PFE)

The Covariance matrix:

|  |  |  |
| :-----| ----: | :----: |
| INTC | 46.0 | 1.06 |
| PFE | 1.06 | 5.33 |

Assume that Intel has a market capitalization of approximately USD 80 Billion
and Pfizer is approximately USD 100 Billions.


In [8]:
tickers = ['INTC','PFE']
weights = pd.Series([80/(80+100),100/(80+100)],index=tickers)

s = pd.DataFrame([[46,1.06],[1.06, 5.33]],index=tickers,columns=tickers)* 10E-4
pi = implied_returns(delta=2.5,sigma=s,w=weights)

In [9]:
pi

INTC    0.052583
PFE     0.008581
Name: Implied Returns, dtype: float64

Thus the equilibrium implied returns for Intel are 5.25% and 0.86% for PFE

Let's assume the investors thinks that Intel will return 2% and that Pfizer will be rebounce to 4%.

We can now examine the optimal weights according to the Markowitz precedure.

What would happen if we used these expected returns to compute the Optimal Max Sharpe Ratio Portfolio?

The Max Sharpe Ratio(MSR) Portfolio weights are easily computed in explicit form if there are no constraints on the weights.
$$ W_{MSR} = \frac{\Sigma^{-1} u_{e}}{1^T\Sigma^{-1}u_e} $$
where $u_e$ is the vector of expected excess returns and $\Sigma$ is the variance-covariance matrix.


In [10]:
# for convenience and readability, define the inverse of a dataframe
def inverse(d):
    '''
    Invert the dataframe by inverting the underlying matrix
    '''
    return pd.DataFrame(inv(d.values), index= d.columns, columns=d.index)

In [11]:
def w_msr(sigma,mu,scale=True):
    '''
    Optimal (Tangent/Max Sharpe Ratio) Portfolio weights
    by using the Markowitz Optimization Procedure 
    Mu is the vector of Excess expected Returns
    Sigma must be an N x N martix as DataFrmae and Mu a column vector as a Series
    '''
    w = inverse(sigma).dot(mu)
    if scale:
        w = w/sum(w)
    return w

Recall that the investor expects that Intel will drop to 2% and Pfizer will return 4%.

We can now examine the optimal weights obtained by naively implementing the Markowitz procedure with these expected returns.

In [12]:
mu_exp = pd.Series([.02, .04],index=tickers)
np.round(w_msr(s,mu_exp)*100,2)

INTC     3.41
PFE     96.59
dtype: float64

We can see that Markowitz procedure places an unrealistic weight of more than 96% in Pfizerand less than 4% in Intel.  
But this is completely impractical and no rational investor would make such dramatic bets.

In contrast, let us now find the weights that the Black Litterman procedure would place.

We allow $\Omega$ to be computed automatically, and are willing to use all the other defaults.  
We find the BL weights as follows:

In [13]:
# Absolute view 1 : INTC will return 2%
# Absolute view 2 : PFE  will return 4%
q = pd.Series({'INTC':0.02,'PFE':0.04})

# The Pick Matrix
# For View 2, it is for PFE
p = pd.DataFrame([
    {'INTC':1,'PFE':0},
    {'INTC':0,'PFE':1}
    ])

#Find the Black Litterman Expected Returns
bl_mu , bl_sigma = bl(w_prior=pd.Series({'INTC':.44,'PFE':.56}),sigma_prior=s,p=p,q=q)

In [14]:
bl_mu

INTC    0.037622
PFE     0.024111
dtype: float64

The posterior returns returned by the procedure are clearly weighted between that of the equilibrium implied expected returns (in the range of 5% and 1%) and that of the investor (2% and 4%).  
The question is are these weights likely to yield more realistic portfolios?  
To answer that question we supply the BL expected returns and covariance martix to the optimizer

In [15]:
bl_sigma

,INTC,PFE
INTC,0.046459,0.001065
PFE,0.001065,0.005383


In [16]:
# Use the Black Litterman expected returns to get the Optimal Markowitz weights
w_msr(bl_sigma,bl_mu)

INTC    0.140692
PFE     0.859308
dtype: float64

We see that we get much more reasonable weights than we did with naive optimization.

## A Simple Example : Relative Views

In this example, we examine rlative views. We stick with our simple 2 stock example.  
Recall that the Cap-Weighted implied expected returns are:

In [17]:
# Expected returns inferred from the cap-weights
pi

INTC    0.052583
PFE     0.008581
Name: Implied Returns, dtype: float64

Recall also that the cap-weighted portfolio is approximately a 45-55 mix of Intel and Pfizer.  

Assume instead that the investor feels that the <font color=#FF0000>Intel will ourperform Pfizer by only 2%.

In [18]:
# Relative View 1: INTC will outperform PFE by 2%
q = pd.Series([
    0.02
])

# The Pick Matrix
p = pd.DataFrame([
    # For View 1, this is for INTC outperforming PFE
    {'INTC':+1, 'PFE':-1}
])

# Find the Black Litterman Expected Returns
bl_mu , bl_sigma = bl(w_prior=pd.Series({'INTC':.44,'PFE':.56}),sigma_prior=s,p=p,q=q)

In [19]:
# New set of BL return
bl_mu

INTC    0.041374
PFE     0.009646
dtype: float64

In [20]:
# The outperformance of Intel in the implied returns is:
pi[0]-pi[1]

0.04400277777777777

In [21]:
# In contrast, the investor felt it only would be 2%.
# The expected returns returned by the BL show a spread
# that is a blend between the cap-weight implied returns and that of the investor:
bl_mu[0]-bl_mu[1]

0.031728

And the weights in the Optimized portfolio when we use these expected returns are:

In [22]:
# Use the Black Litterman expected returns and covariance matrix
w_msr(bl_sigma, bl_mu)

INTC    0.347223
PFE     0.652777
dtype: float64

These seem like more reasonable weights compared to the orignial one.  

In constrast, consider the weights we would get if we implemented the same view without BL.  
We set<font color=#0000FF> the returns of Intel and Pfizer to be 3% and 1% respectively</font>.

In [23]:
w_msr(s,[0.03,0.01])

INTC    0.258528
PFE     0.741472
dtype: float64

The weights are significantly more dramatic than one might be willing to implement, and are likely unwarranted given the realtively weak view.  
In fact, if the same view were implemented as <font color=#0000FF>Intel and Pfizer retruning 2% and 0%</font>, the results are even more extreme:

In [24]:
w_msr(s,[0.02,0.0])

INTC    1.248244
PFE    -0.248244
dtype: float64

In this case, the Markowitz recommends<font color=#0000FF> shorting Pfizer to 24.8% </font>of the portfolio and <font color=#0000FF>leveraging Intel to almost 125%</font>.   

Clearly this is not a sound allocation based on the simple view expressed above.

## He-Litterman 

The He-Litterman example involves an international allocation between 7 countries:

![EX1.jpeg](attachment:EX1.jpeg)

In [25]:
# The 7 countries
countries = ['AU','CA','FR','DE','JP','UK','US']

# Correlation Matrix
rho = pd.DataFrame([
    [1.000,0.488,0.478,0.515,0.439,0.512,0.491],
    [0.488,1.000,0.644,0.655,0.310,0.608,0.779],
    [0.478,0.664,1.000,0.861,0.355,0.783,0.668],
    [0.515,0.655,0.861,1.000,0.354,0.777,0.653],
    [0.439,0.310,0.355,0.354,1.000,0.405,0.306],
    [0.512,0.608,0.783,0.777,0.405,1.000,0.652],
    [0.491,0.779,0.668,0.653,0.306,0.652,1.000]
], index=countries,columns=countries)

# Volatilities
vols = pd.DataFrame([0.160,0.203,0.248,0.271,0.210,0.200,0.187],index=countries,columns=['vol'])

# Cap-Weights
w_eq = pd.DataFrame([0.016,0.022,0.052,0.055,0.116,0.124,0.615], index=countries, columns=['CapWeight'])

# Compute the Covariance Matrix
sigma_prior = vols.dot(vols.T)*rho

# Compute Pi and compare:
pi = implied_returns(delta=2.5, sigma=sigma_prior, w=w_eq)
pi.round(3)

AU    0.039
CA    0.069
FR    0.084
DE    0.090
JP    0.043
UK    0.068
US    0.076
Name: Implied Returns, dtype: float64

![Ex2.jpeg](attachment:Ex2.jpeg)

### View 1: Germany vs Rest of Europe

Next, we impose the view that <font color=#0000FF>German equities will outperform the rest of Europes equities by 5%  

The other European equities are France and the UK. We split the outperformance proportional to the Market Caps of France and the UK.

In [26]:
# Germany will outperform other European Equities (i.e. FR and UK) by 5%
q = pd.Series([.05])

# Start with a single view, all zeors and overwrite the specific view
p = pd.DataFrame([0.]*len(countries),index=countries).T

# find the relative market caps of FR and UK to split the relative outperformance of DE
w_fr = w_eq.loc['FR']/(w_eq.loc['FR']+w_eq.loc['UK'])
w_uk = w_eq.loc['UK']/(w_eq.loc['FR']+w_eq.loc['UK'])
p.iloc[0]['DE']= 1.0
p.iloc[0]['FR']= -w_fr
p.iloc[0]['UK']= -w_uk
(p*100).round(1)

,AU,CA,FR,DE,JP,UK,US
0,0.0,0.0,-29.5,100.0,0.0,-70.5,0.0


In [27]:
table3 = pd.DataFrame(index=pi.index)
table3['p']=p.T.round(3)*100

In [28]:
# Figure out u_bl:
delta = 2.5
tau = 0.05

bl_mu, bl_sigma = bl(w_eq, sigma_prior,p,q,tau=tau)
print((bl_mu*100).round(1))
table3['u_bar']=(bl_mu*100).round(1)

AU     4.3
CA     7.6
FR     9.3
DE    11.0
JP     4.5
UK     7.0
US     8.1
dtype: float64


In [29]:
# Coumpute the optimal portfolio weights
def w_star(delta, sigma, mu):
    return(inverse(sigma).dot(mu))/delta

wstar = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
print((wstar*100).round(1))
table3['w*']=(wstar*100).round(1)

AU     1.5
CA     2.1
FR    -4.0
DE    35.4
JP    11.0
UK    -9.5
US    58.6
dtype: float64


Show the difference between the optimal portfolio and the equilibrium portfolio

In [30]:
w_eq = w_msr(delta*sigma_prior, pi, scale=False)

# Display the difference in posterior and prior weights
print(np.round(wstar-w_eq/(1+tau),3)*100)
table3['w*-w_eq/(1+r)']=np.round(wstar-w_eq/(1+tau),3)*100

AU     0.0
CA    -0.0
FR    -8.9
DE    30.2
JP    -0.0
UK   -21.3
US    -0.0
dtype: float64


In [31]:
table3

,p,u_bar,w*,w*-w_eq/(1+r)
AU,0.0,4.3,1.5,0.0
CA,0.0,7.6,2.1,-0.0
FR,-29.5,9.3,-4.0,-8.9
DE,100.0,11.0,35.4,30.2
JP,0.0,4.5,11.0,-0.0
UK,-70.5,7.0,-9.5,-21.3
US,0.0,8.1,58.6,-0.0


we can tell that those weights for assets that do not involve the view remain unchanged.  

The two underperforming countires (according to the view) are underweighted,   
while the overperforming country is overweighted.

![Ex4.jpeg](attachment:Ex4.jpeg)

## View 2: Canada vs US

For second case, He and Litterman implement the additional view that  
<font color=#0000FF>Canadian Equities will outperform US Equities by 3%

In [32]:
table5=pd.DataFrame(index=countries)

In [33]:
view2 = pd.Series([.03], index=[1])
q = q.append(view2)
pick2 = pd.DataFrame([0.0]*len(countries), index=countries, columns=[1]).T
p = p.append(pick2)
p.iloc[1]['CA']=+1
p.iloc[1]['US']=-1
np.round(p.T,3)*100

,0,1
AU,0.0,0.0
CA,0.0,100.0
FR,-29.5,0.0
DE,100.0,0.0
JP,0.0,0.0
UK,-70.5,0.0
US,0.0,-100.0


In [34]:
table5['P']=p.iloc[0].round(3)*100
table5['Confidence Level']=p.iloc[1]*100

In [35]:
# now compute the BL weights
bl_mu, bl_sigma = bl(w_eq, sigma_prior, p, q, tau=tau)
print(np.round(bl_mu*100, 1))
table5['u_bar']=np.round(bl_mu*100, 1)

AU     4.4
CA     8.7
FR     9.5
DE    11.2
JP     4.6
UK     7.0
US     7.5
dtype: float64


In [36]:
wstar = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
# display w*
print((wstar*100).round(1))
table5['w*']=(wstar*100).round(1)

AU     1.5
CA    41.8
FR    -3.4
DE    33.6
JP    11.0
UK    -8.2
US    18.9
dtype: float64


In [37]:
# Again, show the difference in posterior and prior weights
w_eq = w_msr(delta*sigma_prior, pi, scale=False)
print(np.round(wstar-w_eq/(1+tau), 3)*100)
table5['w*-w_eq/(1+r)']=np.round(wstar-w_eq/(1+tau), 3)*100

AU    -0.0
CA    39.7
FR    -8.4
DE    28.4
JP     0.0
UK   -20.0
US   -39.7
dtype: float64


In [38]:
table5

,P,Confidence Level,u_bar,w*,w*-w_eq/(1+r)
AU,0.0,0.0,4.4,1.5,-0.0
CA,0.0,100.0,8.7,41.8,39.7
FR,-29.5,0.0,9.5,-3.4,-8.4
DE,100.0,0.0,11.2,33.6,28.4
JP,0.0,0.0,4.6,11.0,0.0
UK,-70.5,0.0,7.0,-8.2,-20.0
US,0.0,-100.0,7.5,18.9,-39.7


Once again, the weights for assets that do not involve the view(AU,JP) remain unchanged.  
The two underperforming countries(FR, UK, US, according to the view) are underweighted,    
while the overperforming countries(CA, DE) are overweighted,    
but not to the extreme extent that a naive porfolio optimizer would have produced.

![Ex5.jpeg](attachment:Ex5.jpeg)

## View 3: More Bullish Canada vs US

For their third case, He and Litterman alter the second view that Canadian Equities will outperform US Equities by increasing the expected outperformance from the previously stated 3% to 4%.  

In [39]:
q[1]=0.04
q

0    0.05
1    0.04
dtype: float64

Note that P remains unchanged since we have only altered Q, not P

In [40]:
np.round(p.T*100,1)

,0,1
AU,0.0,0.0
CA,0.0,100.0
FR,-29.5,0.0
DE,100.0,0.0
JP,0.0,0.0
UK,-70.5,0.0
US,0.0,-100.0


In [41]:
# Let examine the black litterman expected returns first
bl_mu , bl_sigma = bl(w_eq, sigma_prior, p, q, tau=tau)
np.round(bl_mu,3)*100

AU     4.4
CA     9.1
FR     9.5
DE    11.3
JP     4.6
UK     7.0
US     7.3
dtype: float64

In [42]:
# We can now compute the Black Litterman weights 
wstar = w_star(delta=2.5,sigma=bl_sigma, mu=bl_mu)
# display w*
(wstar*100).round(1)

AU     1.5
CA    53.3
FR    -3.3
DE    33.1
JP    11.0
UK    -7.8
US     7.4
dtype: float64

![Ex6.jpeg](attachment:Ex6.jpeg)

## View 4: Increasing View Uncertainty

As a final step, He and Litterman demonstrate the effect of $\Omega$.    
They increase the uncertainty associated with the first of the two views    
(i.e. the one that Germany will outperform the rest of Europe).   
First we coumpute the default value of $\Omega$ and then increase the uncertainty associated with the first view alone.

In [43]:
# This is the default 'Proportional to Prior' assumption
omega= proportional_prior(sigma_prior, tau, p)

# Now double the uncertainty associated with View 1
omega.iloc[0,0] = 2*omega.iloc[0,0]
np.round(p.T*100,1)

,0,1
AU,0.0,0.0
CA,0.0,100.0
FR,-29.5,0.0
DE,100.0,0.0
JP,0.0,0.0
UK,-70.5,0.0
US,0.0,-100.0


In [46]:
omega

,0,1
0,0.00213,0.000000
1,0.00000,0.000852


In [44]:
bl_mu, bl_sigma = bl(w_eq, sigma_prior, p, q, tau=tau, omega=omega)

# the black litterman expected returns
np.round(bl_mu,3)*100

AU     4.3
CA     8.9
FR     9.3
DE    10.6
JP     4.6
UK     6.9
US     7.2
dtype: float64

In [45]:
wstar = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
#display w*
(wstar*100).round(1)

AU     1.5
CA    53.9
FR    -0.5
DE    23.6
JP    11.0
UK    -1.1
US     6.8
dtype: float64

Again, we see how the weights increase allocations consistent with the view, but keep allocation from getting extreme

![Ex7.jpeg](attachment:Ex7.jpeg)